# Goal of this File
Create a json file `image_embeddings.json` of the following format
```
{
  'movie_id': {
      'clip': Float[],
      'resnet50': Float[]
  }
}
```
which contains the representations of a movie poster.

In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


FOLDERNAME = 'cs229_proj/'


assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1FYI1gIgILgT31h2y_QMmBZOwkDqTPrg-/cs229_proj


In [ ]:
!ls
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import json
import numpy as np
from PIL import Image # for viewing images
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import time
import copy
from collections import defaultdict

import torch
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from poster_dataset import PosterDataset
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Uses folder and movie_id to create path where image can be found and displayed

def get_poster_path(movie_id, movie_data):
    folder_num = str(movie_data[3]) + "/"
    img_name = movie_id + '.jpg'
    return '/content/drive/My Drive/cs229_proj/letterboxd_posters/' + folder_num + img_name

In [ ]:
movies_full_path = '/content/drive/My Drive/cs229_proj/movies_full.csv'

movies_full_df = pd.read_csv(movies_full_path)
# movies_full_df

In [ ]:
# Resnet50 Embeddings
from torchvision.models import resnet50
resnet_model = resnet50(pretrained=True)
for param in resnet_model.parameters():
    param.requires_grad = False
resnet_model.eval()

resnet_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
resnet_dataset = PosterDataset('letterboxd_posters/', movies_full_df, range(len(movies_full_df)), resnet_transform)

In [ ]:
'''
# sanity check resnet
output = resnet_model(resnet_dataset[0][0].unsqueeze(0))
probabilities = torch.nn.functional.softmax(output[0], dim=0)
plt.plot(probabilities.log())
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())
'''

In [ ]:
# CLIP:
import clip
clip_model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
clip_dataset = PosterDataset('letterboxd_posters/', movies_full_df, range(len(movies_full_df)), preprocess)

In [ ]:
embeddings = defaultdict(dict)
for i in tqdm(range(len(movies_full_df))):
  movie_id = movies_full_df['movie_id'][i]
  embeddings[movie_id]['resnet'] = resnet_model(resnet_dataset[i][0].unsqueeze(0))[0].tolist()
  embeddings[movie_id]['clip'] = clip_model.encode_image(clip_dataset[i][0].unsqueeze(0)).reshape(512,).tolist()

100%|██████████| 2220/2220 [23:50<00:00,  1.55it/s]


In [ ]:
with open('embeddings.json', 'w') as fp:
    json.dump(embeddings, fp)